In [ ]:
import os
import PIL
import torch as T
import numpy as np
import pandas as pd
from PIL import Image
import cv2

import torch
from torch.utils.data import Dataset
from torchvision.datasets import DatasetFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
import torchmetrics
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import torchvision.models as models
from torch.optim.lr_scheduler import StepLR


### Для получения labels из названий папок

In [ ]:
class CustomDateset(Dataset):
    def __init__(self, csv_data_path):
        
        self.df = pd.read_csv(csv_data_path)
        self.transforms = transforms.Compose([transforms.ToTensor()])
                 # transforms.Normalize(mean = (0.5), std = (0.5))])
         
    def __len__(self):
        return (len(self.df))
    
    def __getitem__(self, i):
        
        image = cv2.imread(self.df.loc[i,'path'])        
        label = self.df.loc[i,'lable']
        
        if self.transforms:
            image = self.transforms(image)
    
        return image, label # , scalar


In [ ]:
train_data = CustomDateset('./train.csv')
val_data = CustomDateset('./val.csv')

trainloader = DataLoader(train_data, batch_size=10, shuffle=True, num_workers = 0)
valloader = DataLoader(val_data, batch_size=10, shuffle=True, num_workers = 0)

In [ ]:

def train(trainloader, valloader, n_epochs, model = None):
    #l2 regular parameter
    l2_lambda = 0.001
    # num classes
    num_classes = 8
    # learning rate
    lr = 0.01

    # loss function
    criterion = nn.CrossEntropyLoss()
    # device
    device = T.device('cuda' if T.cuda.is_available() else 'cpu')
    if model is None:
        model = models.resnet18(num_classes = 8).to(device)
    # parallel device learning learning
    # model= nn.DataParallel(model,device_ids = [0,1,2])

    # metrics 
    Acc = torchmetrics.classification.MulticlassAccuracy(num_classes = 8, average='micro').to(device)
    Precision = torchmetrics.classification.MulticlassPrecision(num_classes = 8, average=None).to(device)
    Recall = torchmetrics.classification.MulticlassRecall(num_classes = 8, average=None).to(device)
    
    writer = SummaryWriter()
    # optimizer and lr step
    optimizer = optim.RMSprop(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

    for epoch in range(n_epochs):  # no. of epochs
        # Train 
        running_train_loss = 0
        running_train_precision = 0
        running_train_recall = 0
        running_train_acc = 0
        
        model.train()
        
        for data in tqdm(trainloader, ncols=100):
            # data pixels and labels to GPU if available
            inputs = data[0].to(device, non_blocking=True)
            labels = data[1].to(device, non_blocking=True)

            # set the parameter gradients to zero
            optimizer.zero_grad()
            outputs = model.forward(inputs)
            
            loss = criterion(outputs, labels)
            outputs = T.argmax(F.softmax(outputs, dim=1), dim=1)
            
            # outputs,labels = outputs, labels.cpu().numpy()

            precision = Precision(outputs, labels)
            recall    = Recall(outputs, labels)
            acc       = Acc(outputs, labels)
            
            # l2 regularization cumpute 
            l2_reg = torch.tensor(0.).to(device)
            for param in model.parameters():
                l2_reg += torch.norm(param)
            loss += l2_lambda * l2_reg
            
            # propagate the loss backward
            loss.backward()
            # update the gradients
            optimizer.step()
            
            running_train_loss += loss.item()
            running_train_precision += precision.cpu().numpy()
            running_train_recall += recall.cpu().numpy()
            running_train_acc += acc.item()
                
        # Validation
        running_val_loss = 0
        running_val_precision = 0
        running_val_recall = 0
        running_val_acc = 0
            
        model.eval()
            
        with T.no_grad():
            for data in tqdm(valloader, ncols=100):
                # data pixels and labels to GPU if available
                inputs = data[0].to(device, non_blocking=True)
                labels = data[1].to(device, non_blocking=True)
    
                # set the parameter gradients to zero
                outputs = model.forward(inputs)
                
                loss = criterion(outputs, labels)
                outputs = T.argmax(F.softmax(outputs, dim=1), dim=1)
                
                # outputs,labels = outputs.cpu().numpy(), labels.cpu().numpy()

                precision = Precision(outputs, labels)
                recall    = Recall(outputs, labels)
                acc       = Acc(outputs, labels)
                
                running_val_loss += loss.item()
                running_val_precision += precision.cpu().numpy()
                running_val_recall += recall.cpu().numpy()
                running_val_acc += acc.item()
                
        scheduler.step()
        
        writer.add_scalar("Train/acc", running_train_acc / len(trainloader), epoch + 1)
        writer.add_scalar("Train/precision", running_train_precision.mean() / len(trainloader), epoch + 1)
        writer.add_scalar("Train/recall", running_train_recall.mean() / len(trainloader), epoch + 1)
        
        writer.add_scalar("Train/loss", running_train_loss / len(trainloader), epoch + 1)
        
        writer.add_scalar("Parameters/Learning Rate", optimizer.param_groups[0]['lr'], epoch + 1)

        writer.add_scalar("Val/acc", running_val_acc/ len(valloader), epoch + 1)
        writer.add_scalar("Val/precision", running_val_precision.mean() / len(valloader), epoch + 1)
        writer.add_scalar("Val/recall", running_val_recall.mean() / len(valloader), epoch + 1)
        writer.add_scalar("Val/loss", running_val_loss / len(valloader), epoch + 1)
    
        print(f' Epoch       {epoch + 1}\n', 
                f'loss:      {running_train_loss / len(trainloader)}\n',
                f'precision: {running_train_precision / len(trainloader)}\n',
                f'recall:    {running_train_recall / len(trainloader)}\n',
                f'acc:       {running_train_acc / len(trainloader)}\n')
        print('-------------------------------------------')
        print(f'Validation  \n', 
                f'loss:      {running_val_loss / len(valloader)}\n',
                f'precision: {running_val_precision / len(valloader)}\n',
                f'recall:    {running_val_recall / len(valloader)}\n',
                f'acc:       {running_val_acc / len(valloader)}\n')
        print('-------------------------------------------')
        # save model
        torch.save(model, './res18_lr' + str(epoch))

In [ ]:
train(trainloader, valloader, n_epochs = 10)

In [ ]:
# model = models.resnet18(num_classes = 8)
# torch.save(model, './net_learned_on_Medvedev_dataset.pth')

In [ ]:
# !tensorboard --logdir=runs